Consider the problem of brute-force searching.

Define $f:\{0, 1\}^{n} \rightarrow \{0, 1\}$ such that $f(x)=1$ for a specific input $w \in \{0, 1\}^{n}$, and $f(x)=0$ otherwise.

The task is to find the value of $w$.

Classically, in the worst case, solving this problem requires $N=2^{n}$ queries. On average, $N/2$ queries are needed.

As a quantum solution, Grover's algorithm can solve this problem using $O(\sqrt{N})$ queries.

In [ ]:
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt

n = 5

dev_count = qml.device("default.qubit", wires=n+1, shots=10000)
dev_prob = qml.device("default.qubit", wires=n+1)

-- Grover's algorithm --

(1) Prepare $\left|0\right>^{\otimes n}$ and perform $H^{\otimes n}$ to the state.  
(2) Apply the oracle operator $U_{f}$.  
(3) Execute the reflection gate (also known as the Grover diffusion operator).  
(4) Repeat steps (2) and (3) until the number of iterations $t$ reaches $t \approx \pi \sqrt{N} /4$.  
(5) Perform measurement.

In [ ]:
# oracle operator example
def oracle(bitstring_w):
    n = len(bitstring_w)
    for i in range(n):
        if bitstring_w[i] == '0':
            qml.PauliX(wires=i+1)
    qml.MultiControlledX(wires=[i+1 for i in range(n)]+[0])
    for i in range(n):
        if bitstring_w[i] == '0':
            qml.PauliX(wires=i+1)


# oracle oprator figure
fig, ax = qml.draw_mpl(oracle)('01011')
plt.show()

Reflection operator: $R_{s}=2 \left|s\right>\left<s\right| -I$, where $\left|s \right>=H^{\otimes n}\left|0\right>^{\otimes n}$.  
$\Rightarrow$ $R_{s}=H^{\otimes n}R_{0}H^{\otimes n}$, where $R_{0}=2 \left|0\right>\left<0\right|^{\otimes n} -I$.


In [ ]:
# reflection gate
def reflection():
    for i in range(n):
        qml.Hadamard(wires=i+1)
    qml.Barrier()
    for i in range(n):
        qml.PauliX(wires=i+1)
    # HXH=Z
    qml.Barrier()
    qml.Hadamard(wires=n)
    qml.MultiControlledX(wires=[i+1 for i in range(n-1)]+[n])
    qml.Hadamard(wires=n)
    qml.Barrier()
    for i in range(n):
        qml.PauliX(wires=i+1)
    qml.Barrier()
    qml.PauliX(wires=n)
    qml.PauliZ(wires=n)
    qml.PauliX(wires=n)
    qml.PauliZ(wires=n)
    qml.Barrier()
    for i in range(n):
        qml.Hadamard(wires=i+1)

# reflection gate figure
fig, ax = qml.draw_mpl(reflection)()
plt.show()

In [ ]:
# Grover's algorithm

import random
random_bitstring_w = format(random.getrandbits(n), 'b').zfill(n)

# measurement: counts
@qml.qnode(dev_count)
def Grover_algorithm_count(n_query):
    qml.PauliX(wires=0)
    for i in range(n+1):
        qml.Hadamard(wires=i)
    for t in range(n_query):
        oracle(random_bitstring_w)
        reflection()
    return qml.counts(wires=[i for i in range(1, n+1)], all_outcomes=True)

# measurement: probability
@qml.qnode(dev_prob)
def Grover_algorithm_prob(n_query):
    qml.PauliX(wires=0)
    for i in range(n+1):
        qml.Hadamard(wires=i)
    qml.Barrier(wires=[i for i in range(n+1)])
    for t in range(n_query):
        oracle(random_bitstring_w)
        qml.Barrier(wires=[i for i in range(n+1)])
        reflection()
        qml.Barrier(wires=[i for i in range(n+1)])
    return qml.probs(wires=[i for i in range(1, n+1)])

In [ ]:
fig, ax = qml.draw_mpl(Grover_algorithm_prob)(2)
plt.show()

As shown in the graphs below, by applying Grover's algorithm, we can find the desired element $w$ with high probability.  
Note: If the number of iterations exceeds a certain point, the probability of finding $w$ starts to decrease again.

In [ ]:
print('desired value w:', random_bitstring_w)
bitstring_list = [format(i, 'b').zfill(n) for i in range(2**n)]

for t in range(8):
    result = Grover_algorithm_prob(n_query=t)
    print('result:', result)
    plt.bar(bitstring_list, result)
    plt.xticks(rotation="vertical")
    plt.xlabel('Bitstring')
    plt.ylabel('Probability')
    plt.title('n_query={}'.format(t))
    plt.show()

In [ ]:
print('desired value w:', random_bitstring_w)

for t in range(8):
    print('result:', Grover_algorithm_count(n_query=t))
    keys = list(Grover_algorithm_count(n_query=t).keys())
    values = list(Grover_algorithm_count(n_query=t).values())
    plt.bar(keys, values)
    plt.xticks(rotation="vertical")
    plt.xlabel('Bitstring')
    plt.ylabel('Counts')
    plt.title('n_query={}'.format(t))
    plt.show()

-- Reference --   
Lov K. Grover, "A fast quantum mechanical algorithm for database search." Proceedings of the twenty-eighth annual ACM symposium on Theory of computing (1996).